In [4]:
import os
import shutil
import stat
from PIL import Image

In [5]:
def convert_png(directory, label, target):
    for i, image in enumerate(os.listdir(directory)):
        f = os.path.join(directory, image)
        if os.path.isfile(f):
            image = Image.open(f)
            image.save(f'data/{i}{label}.{target}')
            
    os.system('rmdir /S /Q "{}"'.format(directory))

In [6]:
"""
    DO NOT RUN
"""

directory_1 = 'data/cancer'
directory_2 = 'data/non-cancer'
target = '.png'

convert_png(directory_1, 1, target)
convert_png(directory_2, 0, target)